In [60]:
!pip install flatten_json

  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7967 sha256=f4bde9e69d84bdadaa29aa503c87ab3b7affd26febe98ab4325117cb1fa46898
  Stored in directory: /Users/sample/Library/Caches/pip/wheels/e6/b3/2a/beb2ceb72d11bf335f9c2f87aae26981f6744f3fc885cde665
Successfully built flatten-json


In [1]:
import pandas as pd
import requests
import json
from flatten_json import flatten
import re

url = "https://run.mocky.io/v3/15dd8916-0427-4c82-934f-a8d1d105e4ec"
resp = requests.get(url=url)
data = resp.json()

dict_flattened = (flatten(record, '.',root_keys_to_ignore={'SERA_COUNT', 'GNI_COUNT'}) for record in data)
df = pd.DataFrame(dict_flattened)


In [ ]:
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(inplace=True, axis=1, how='all')

In [ ]:
df

In [2]:
df = df.T.drop_duplicates().T
df

SYS_ID SYS_NAME SERA_RECORDS.0.SYS_ID SERA_RECORDS.1.OWNER  \
0    123    File1                                        YES   
1    124    File1                                        YES   

  GNI_RECORDS.1.CLASSIFICATION CITY REGION  
0                      NETWORK  JSQ     US  
1                       server  lon     Uk

In [3]:
df.rename(columns=lambda x: re.sub(r'^(.*\.(\d)\.)',r'',x), inplace=True)
df.to_csv('file_name.csv', encoding='utf-8')
!cat file_name.csv


,SYS_ID,SYS_NAME,SYS_ID,OWNER,CLASSIFICATION,CITY,REGION
0,123,File1,,YES,NETWORK,JSQ,US
1,124,File1,,YES,server,lon,Uk


In [4]:
import sys
import csv
import re
import datetime
import logging
import string

from netaddr import IPNetwork, IPSet
from argparse import ArgumentParser
from collections import defaultdict
from getpass import getpass
from time import sleep
from typing import Dict, List, Tuple, Any, Set
from contextlib import contextmanager
from pathlib import Path

from api.guardicore import RESTManagementAPI, ManagementAPIError, ManagementAPITimeoutError
from common.labels.utils import Label, NameCriterion, StringIsNotNameCriterion, CHARS_ILLEGAL_IN_LABELS
from common.common import validate_python_version, initiate_logger


In [5]:
EXPORT_COMMAND_INFORMATIONAL_HEADERS = ["Comments", "Asset Status", "IP Addresses", "Asset ID"]
# GC API Settings
ASSET_OBJECTS_TO_GET_AT_ONCE = 1000
LABEL_OBJECTS_TO_GET_AT_ONCE = 1000
LABEL_OBJECTS_TO_GET_AT_ONCE_AFTER_TIMEOUT = 50
DYNAMIC_CRITERIA_LIMIT = 500000
# IP address with / without CIDR regex
IP_REGEX = r"^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])" \
           r"(/([0-9]|[1-2][0-9]|3[0-2]))?$"
# Subnets reserved for special usage, alert the user on the attempt to upload those as dynamic criteria
RESERVED_SUBNETS = IPSet(['224.0.0.0/3', "127.0.0.0/8", "0.0.0.0/8"])

DYNAMIC_TAG = ' <assigned dynamically>'

# Increase csv max field size to handle large label comments + for restore command
DEFAULT_CSV_FIELD_SIZE = csv.field_size_limit()
csv.field_size_limit(DEFAULT_CSV_FIELD_SIZE*1000)

131072

In [66]:
@contextmanager
def initiate_gc_api():
    """
    A contextmanager that yields a Centra API connection object (RESTManagementAPI) using credentials provided via
    command line or via an ini file. An attempt to log out from Centra API will be preformed when the object is API
    connection is no longer needed.
    """
    # Get configuration from ini file
    logger.info(f"Reading centra environment details from the ini file")
    try:
        config = ConfigParser(interpolation=None)
        with Path('gc_labeler.ini').open("r") as conf_file:
            config.read_file(conf_file)
            try:
                management_address = config.get("gc_api", "management_address")
                management_username = config.get("gc_api", "username")
                management_password = config.get("gc_api", "password")
                management_port = config.get("gc_api", "port")
            except (NoSectionError, NoOptionError, MissingSectionHeaderError) as error:
                    logger.error(f"Error while reading the configuration file: {repr(error)}")
                    logger.info("Aborting..")
                    sys.exit(1)
    except IOError as error:
                    logger.error(f"Error while reading the ini file: {repr(error)}")
                    logger.info(f"Make sure you copied gc_labeler-sample.ini file to '{Path(args.ini_file_path)}'")
                    logger.info("Aborting..")
                    sys.exit(1)
    try:
        if "https" in management_address:
            logger.error(f"The provided management address {management_address} contains 'https', which should "
                         f"not be provided in this field. Please provide only the ip (i.e. 172.16.100.1) or fqdn "
                         f"(i.e centra.example.com) of the management server, without https:// or any trailing slashes")
            sys.exit(1)
        logger.info(f"Connecting to Centra API at '{management_address}' over port {management_port}, using the "
                    f"username {management_username}")
        gc_api = RESTManagementAPI(management_host=management_address, allow_2fa_auth=True,
                                   username=management_username, password=management_password,
                                   port=management_port)
        yield gc_api
    except ManagementAPIError as error:
        logger.error(f"Could not connect to Centra API: {repr(error)}")
        logger.info("Aborting..")
        sys.exit(1)
    finally:
        try:
            logger.debug(f"Trying to logout properly from Centra API")
            gc_api.logout()
        except NameError:
            pass
        except Exception as error:
            logger.debug(f"Could not Trying to logout properly from Centra API. {repr(error)}")

In [51]:
def import_labels_from_csv():
    """
    Import labels main function.
    """
    raw_labels_to_import = parse_labels_csv(args.labels_file_path, args.ignore_labels_export_metadata,
                                            args.allow_duplicates)
    raw_labels_to_import_for_assets, raw_labels_to_import_for_subnets, labels_to_import_for_name_criteria = \
        sort_labels_for_targets_by_target_types(raw_labels_to_import, args.ignore_name_criteria)

    with initiate_gc_api() as gc_api:
        centra_assets_raw = get_centra_assets(gc_api, include_deleted_assets=args.include_deleted_assets)
        centra_assets = {asset["id"]: asset for asset in centra_assets_raw}

        centra_labels_raw = get_centra_labels(gc_api)
        centra_labels = {Label(label_obj["key"], label_obj["value"]): label_obj for label_obj in centra_labels_raw if
                         not is_orchestration_label(label_obj['key'])}

        labels_to_import_for_assets, duplicate_labels_for_assets, assets_not_in_centra = \
            process_raw_labels_to_import_for_assets(raw_labels_to_import_for_assets, centra_labels, centra_assets)

        labels_to_import_for_subnets, duplicate_labels_for_subnets = \
            process_raw_labels_to_import_for_subnets(raw_labels_to_import_for_subnets, centra_labels,
                                                     args.treat_subnets_as_networks)

        labels_to_import_for_name_criteria, duplicate_labels_for_name_criteria = \
            process_raw_labels_to_import_for_name_criteria(labels_to_import_for_name_criteria, centra_labels)

        # A dictionary to store the new labels to create. Structure: {Label: criteria}
        centra_labels_to_create = defaultdict(dict)
        # A set to store all the labels that need to be modified
        centra_labels_to_update = set()
        # A list to store the actions to be taken as strings.
        actions_as_strings = list()

        labels_to_remove_assets_from = \
            generate_import_and_update_actions_for_assets(labels_to_import_for_assets, duplicate_labels_for_assets,
                                                          args.allow_duplicates, centra_labels, centra_assets,
                                                          centra_labels_to_create, centra_labels_to_update,
                                                          actions_as_strings)

        generate_import_and_update_actions_for_subnets(labels_to_import_for_subnets,
                                                       duplicate_labels_for_subnets, args.allow_duplicates,
                                                       centra_labels, centra_labels_to_create, centra_labels_to_update,
                                                       actions_as_strings, args.treat_subnets_as_networks)

        generate_import_and_update_actions_for_name_criteria(labels_to_import_for_name_criteria,
                                                             duplicate_labels_for_name_criteria,
                                                             args.allow_duplicates, centra_labels,
                                                             centra_labels_to_create, centra_labels_to_update,
                                                             actions_as_strings)

        env_name = gc_api.get_env_name()

        num_of_labels_with_updates = len(centra_labels_to_update | set(labels_to_remove_assets_from.keys()))
        if print_summary_and_ask_for_confirmation_for_import_action(len(centra_labels_to_create),
                                                                    num_of_labels_with_updates, actions_as_strings,
                                                                    assets_not_in_centra, env_name):
            if centra_labels_to_create:
                upload_new_labels_to_centra(gc_api, centra_labels_to_create)
            if centra_labels_to_update:
                update_centra_labels(gc_api, centra_labels_to_update, centra_labels)
            if labels_to_remove_assets_from:
                remove_assets_from_labels(gc_api, labels_to_remove_assets_from)

            logger.info(f"Finished importing labels to {env_name}.")

In [52]:
def parse_labels_csv(labels_file_path: str, ignore_labels_export_metadata: bool,
                     allow_duplicates: bool) -> Dict[str, List[Label]]:

    logging.info(f"Reading labels from the CSV file '{labels_file_path}'")
    try:
        with open(labels_file_path, mode="r", encoding="utf8") as csv_file:
            reader = csv.DictReader(csv_file)
            # target_column - the first column in the CSV. Contains the asset names / subnets to add the labels to
            target_column = reader.fieldnames[0]

            # Perform validation on the label keys = the CSV column headers after the target_column header
            for column_number, key in enumerate(reader.fieldnames[1:], 2):
                illegal_characters_in_key = [char for char in CHARS_ILLEGAL_IN_LABELS if char in key]
                if illegal_characters_in_key:
                    logger.error(f"The header of column number {column_number} ('{key}') contains the characters "
                                 f"'{', '.join(illegal_characters_in_key)}' which are illegal for use in labels.")
                    logger.info("Aborting..")
                    sys.exit(1)
                if key in EXPORT_COMMAND_INFORMATIONAL_HEADERS:
                    if ignore_labels_export_metadata:
                        logger.debug(f"ignore_labels_export_metadata is True. Ignoring all the values of the "
                                     f"column header '{key}'.")
                    else:
                        logger.error(f"The header of column number {column_number} ('{key}') is generated by the "
                                     f"Export command. If you wish to ignore metadata generated by the export command, "
                                     f"re run the script with the --ignore_labels_export_metadata flag. \n"
                                     f"Refer to the section 'Using label export to backup the labels in Centra' in the "
                                     f"script's documentation for more info.")
                        logger.info("Aborting..")
                        sys.exit(0)

            raw_labels_for_targets = defaultdict(list)
            for row_number, row in enumerate(reader, 2):
                target = row.pop(target_column).strip()
                for key, values in row.items():  # values can be a single value or a comma separated multiple values
                    if not key:
                        logger.error(f"Row number {row_number} contains too many commas. Please fix the CSV and re "
                                     f"run the script.")
                        logger.info("Aborting..")
                        sys.exit(1)
                    if key in EXPORT_COMMAND_INFORMATIONAL_HEADERS:
                        continue
                    if values is None:
                        continue
                    if ',' in values:
                        # Multiple values were found in the cell
                        if allow_duplicates:
                            values = values.split(',')
                        else:
                            # Multiple values were found in the cell, but it is not allowed
                            logger.error(f"The value '{values}' for the key '{key}' in row {row_number} contains the "
                                         f"comma sign (',') which indicates multiple values, however the "
                                         f"--allow_duplicates flag was not provided. Please refer to the script's "
                                         f"documentation for more info")
                            logger.info("Aborting..")
                            sys.exit(1)
                    else:
                        # A single value in the cell
                        values = [values]
                    for value in values:
                        if value:
                            if DYNAMIC_TAG in value:
                                if ignore_labels_export_metadata:
                                    continue  # Ignore cells containing the dynamic criteria mark
                                else:
                                    logger.warning(f"The value '{value}' for the key '{key}' in row {row_number} "
                                                   f"contains the dynamic tag '{DYNAMIC_TAG}' which is generated by "
                                                   f"the Export command. If you wish to ignore those values, re run "
                                                   f"the script with the --ignore_labels_export_metadata flag. "
                                                   f"Refer to the section 'Using label export to backup the labels in "
                                                   f"Centra' in the script's documentation for more info")
                                    logger.info("Aborting..")
                                    sys.exit(0)
                            illegal_characters_in_value = [char for char in CHARS_ILLEGAL_IN_LABELS if char in value]
                            if illegal_characters_in_value:
                                logger.error(f"The value '{value}' for the key '{key}' in row {row_number} contains "
                                             f"the characters '{', '.join(illegal_characters_in_value)}' which are "
                                             f"illegal for use in labels.")
                                logger.info("Aborting..")
                                sys.exit(1)
                            raw_labels_for_targets[target].append(Label(key=key.strip(), value=value.strip()))
    except IOError as e:
        logger.error(f"Error while reading csv file '{labels_file_path}'. {repr(e)}.")
        logger.info("Aborting..")
        sys.exit(1)
    if raw_labels_for_targets:
        return raw_labels_for_targets
    else:
        logger.info("The CSV does not contain any labels to import or delete. Exiting..")
        sys.exit(0)

In [53]:
def sort_labels_for_targets_by_target_types(raw_labels_for_targets: Dict[str, List[Label]],
                                            ignore_name_criteria: bool) -> \
        Tuple[Dict[str, List[Label]], Dict[IPNetwork, List[Label]], Dict[NameCriterion, List[Label]]]:
    """
    Iterate over raw_labels_for_targets and sort the targets into subnets (match IPv4 ip / subnet), name_criteria, and
    potential Centra assets (The validation whether an asset indeed exists in Centra will be performed later).
    :param ignore_name_criteria: Bool. Whether to treat targets that includes '*' signs as name dynamic criteria or
    as normal asset names
    :param raw_labels_for_targets: a dictionary mapping every target to a list of it's labels to import or delete
    :return:
        labels_for_asset_targets: a dictionary mapping assets to their labels to import or delete
        labels_for_subnet_targets: a dictionary mapping IPNetwork objects to their labels to import
        labels_for_name_criteria_targets: a dictionary mapping name criteria to their labels to import
    """
    labels_for_asset_targets = dict()
    labels_for_subnet_targets = dict()
    labels_for_name_criteria_targets = dict()
    for row_number, (target, labels_to_import_for_target) in enumerate(raw_labels_for_targets.items(), 2):
        if is_subnet_or_ip(target):
            subnet = IPNetwork(target)
            if is_reserved_subnet(subnet):
                logger.warning(f"The subnet {subnet} mentioned in line {row_number} of the CSV is reserved for special"
                               f" uses.")
            labels_for_subnet_targets[subnet] = labels_to_import_for_target
        elif not ignore_name_criteria:
            try:
                name_criterion = NameCriterion.convert_string_to_name_dynamic_criteria_object(target)
                labels_for_name_criteria_targets[name_criterion] = labels_to_import_for_target
            except StringIsNotNameCriterion:
                asset_name = target.lower()  # Asset names are case insensitive
                labels_for_asset_targets[asset_name] = labels_to_import_for_target
        else:
            asset_name = target.lower()  # Asset names are case insensitive
            labels_for_asset_targets[asset_name] = labels_to_import_for_target
    return labels_for_asset_targets, labels_for_subnet_targets, labels_for_name_criteria_targets

In [54]:
def get_centra_labels(gc_api: RESTManagementAPI, filter_keys: List[str] = None,
                      label_objects_to_get_at_once: int = LABEL_OBJECTS_TO_GET_AT_ONCE) -> List[Dict[str, Any]]:
    """
    Query Centra API for all the labels in Centra
    :param label_objects_to_get_at_once: The amount of label objects to request from the API in one call
    :param filter_keys: A list of keys. If provided, request only for labels with one of the provided keys
    :param gc_api: RESTManagementAPI object
    :return: a list containing all Centra label objects
    """
    logging.info("Fetching labels data from Centra")
    centra_labels = list()
    offset = 0
    logger.debug(f"Requesting for chunk of {label_objects_to_get_at_once} labels from Centra")
    try:
        if filter_keys:
            logger.debug(f"Requesting only labels with the following filter_keys: {filter_keys}")
            response = gc_api.list_visibility_labels(limit=label_objects_to_get_at_once,
                                                     dynamic_criteria_limit=DYNAMIC_CRITERIA_LIMIT,
                                                     key=','.join(filter_keys))
        else:
            response = gc_api.list_visibility_labels(limit=label_objects_to_get_at_once,
                                                     dynamic_criteria_limit=DYNAMIC_CRITERIA_LIMIT)
    except ManagementAPITimeoutError:
        logger.warning("The request for labels from Centra has timed out")
        logger.info("Sleeping for 60 seconds and trying again with a lower the number of labels requested at once.")
        label_objects_to_get_at_once = LABEL_OBJECTS_TO_GET_AT_ONCE_AFTER_TIMEOUT
        sleep(60)
        if filter_keys:
            logger.debug(f"Requesting only labels with the following filter_keys: {filter_keys}")
            response = gc_api.list_visibility_labels(limit=label_objects_to_get_at_once,
                                                     dynamic_criteria_limit=DYNAMIC_CRITERIA_LIMIT, key=filter_keys)
        else:
            response = gc_api.list_visibility_labels(limit=label_objects_to_get_at_once,
                                                     dynamic_criteria_limit=DYNAMIC_CRITERIA_LIMIT)
    while len(response["objects"]) > 0:
        for label_obj in response["objects"]:
            try:
                validate_label(label_obj)
                centra_labels.append(label_obj)
            except AssertionError as e:
                logger.warning(f"Invalid label found in Centra: {e}. Please contact Guardicore support")
                logger.debug(label_obj)
        if len(response["objects"]) == label_objects_to_get_at_once:
            offset += LABEL_OBJECTS_TO_GET_AT_ONCE
            logger.debug(f"Requesting {label_objects_to_get_at_once} labels from Centra, with offset {offset}")
            if filter_keys:
                response = gc_api.list_visibility_labels(limit=label_objects_to_get_at_once, offset=offset,
                                                         dynamic_criteria_limit=DYNAMIC_CRITERIA_LIMIT,
                                                         key=','.join(filter_keys))
            else:
                response = gc_api.list_visibility_labels(limit=label_objects_to_get_at_once, offset=offset,
                                                         dynamic_criteria_limit=DYNAMIC_CRITERIA_LIMIT)
        else:
            break
    logger.debug(f"Got {len(centra_labels)} labels from Centra")
    return centra_labels

In [55]:
def is_orchestration_label(key):
    """
    Return True if the label was automatically generated by Centra orchestration else, return False.
    Supported orchestrations are VMWare, AWS and Openshift / K8s

    """
    # vSphere orchestration keys
    if key in {'vCenter folder', 'vCenter host'}:
        return True
    # AWS orchestration keys
    if key in {'AWS Image ID', 'AWS Instance Type', 'AWS Owner ID', 'AWS Availability Zone'}:
        return True
    # K8s orchestration keys
    if key in {'OpenShift Namespace', 'Openshift Service', 'Openshift-io/component', 'Kubernetes-io/hostname',
               'OpenShift Node', 'OpenShift', 'Component', 'Beta-kubernetes-io/arch', 'Pod-template-generation',
               'Node-role-kubernetes-io/infra', 'Provider', 'Openshift-io/control-plane',
               'Node-role-kubernetes-io/master', 'Docker-registry', 'Controller-revision-hash', 'Router'}:
        return True
    # Azure orchestration keys
    if key in {'Azure Resource Group', 'Azure Location'}:
        return True
    return False

In [56]:
def process_raw_labels_to_import_for_assets(raw_labels_to_import_for_assets, centra_labels, centra_assets):
    """
    Process the raw labels to import for assets. For each asset:
    1. Validate whether it is indeed a legal asset in Centra (targets that are actually not assets will be detected
    in this function)
    2. Create a list of all the new labels that should be imported for it (= the asset is not currently explicitly
    labeled with them)
    3. Create a list of all the duplicate labels (multiple labels with the same key) it will have in case the labels
    import action will be performed.
    :param raw_labels_to_import_for_assets: a dictionary mapping every asset to the list of it's labels to import
    :param centra_labels: dict containing all Centra labels ((key, value): label_obj)
    :param centra_assets: dict containing all Centra assets (asset_id: asset_obj)
    :return:
        labels_to_import - dict mapping each asset_id to it's labels to import
        duplicate_labels - dict mapping each asset_id to it's duplicate labels
        assets_not_in_centra - list of assets that were mentioned in the CSV but were not found in Centra
    """
    # Locate assets that were mentioned in the CSV but are missing from Centra
    assets_not_in_centra = set()
    asset_name_to_asset_ids = create_asset_name_to_asset_ids_mapping(centra_assets,
                                                                     set(raw_labels_to_import_for_assets.keys()))
    for potential_asset in raw_labels_to_import_for_assets:
        if potential_asset not in asset_name_to_asset_ids:
            logger.warning(f"The asset {potential_asset} was not found in Centra.")
            assets_not_in_centra.add(potential_asset)
    # Remove non existing assets from raw_labels_to_import_for_assets
    for asset_not_in_centra in assets_not_in_centra:
        del raw_labels_to_import_for_assets[asset_not_in_centra]

    current_explicit_labels_for_assets = map_assets_to_explicit_labels(centra_labels, asset_name_to_asset_ids)
    labels_to_import = defaultdict(list)
    duplicate_labels = dict()
    for asset_name, raw_labels_to_import_for_asset in raw_labels_to_import_for_assets.items():
        # There might be more than one asset with the same name, so it is necessary to iterate over all those
        for asset_id in asset_name_to_asset_ids[asset_name]:
            current_labels_for_asset = current_explicit_labels_for_assets.get(asset_id, list())
            for label_to_import in raw_labels_to_import_for_asset:
                if label_to_import in current_labels_for_asset:
                    logger.debug(f"The asset {asset_name} with the asset_id '{asset_id}' is already labeled with "
                                 f"the label '{label_to_import}'")
                else:
                    labels_to_import[asset_id].append(label_to_import)

            duplicate_labels_for_asset = \
                locate_duplicate_labels_for_asset(asset_id, asset_name, raw_labels_to_import_for_asset,
                                                  current_labels_for_asset)
            if duplicate_labels_for_asset:
                duplicate_labels[asset_id] = duplicate_labels_for_asset

    return labels_to_import, duplicate_labels, assets_not_in_centra

In [57]:
def process_raw_labels_to_import_for_subnets(raw_labels_to_import_for_subnets, centra_labels,
                                             treat_subnets_as_networks):
    """
    Process the raw labels to import for subnets. For each subnet:
    1. Create a list of all the new labels that should be imported for it (= the subnet is not already labeled in
     Centra with this label)
    2. Create a list of all the duplicate labels (multiple labels with the same key) it will have in case the labels
    import action will be performed.
    :param treat_subnets_as_networks: If True, subnets will be treated as network objects. This means that the
    subnet will be counted as if it is already labeled with some label if the label current criteria covers this subnet.
    For example - 10.0.0.1/32 will be counted inside the label App: Ecomm if the App: Ecomm has a dynamic subnet
    criteria of 10.0.0.0/8.
    If False, only exact matches will be counted - 10.0.0.1/32 not already covered by 10.0.0.0/8
    :param raw_labels_to_import_for_subnets: a dictionary mapping every subnet to the list of it's labels to import
    :param centra_labels: dict containing all Centra labels ((key, value): label_obj)
    :return:
        labels_to_import - dict mapping each subnet to it's labels to import
        duplicate_labels - dict mapping each subnet to it's duplicate labels
    """
    labels_to_import = defaultdict(list)
    duplicate_labels = dict()
    if treat_subnets_as_networks:
        current_labels_for_subnets = \
            map_current_labels_for_subnets_as_networks(centra_labels, set(raw_labels_to_import_for_subnets.keys()))
    else:
        current_labels_for_subnets = \
            map_current_labels_for_subnets_as_strings(centra_labels, set(raw_labels_to_import_for_subnets.keys()))

    for subnet, raw_labels_to_import_for_subnet in raw_labels_to_import_for_subnets.items():
        current_labels_for_subnet = current_labels_for_subnets.get(subnet, list())
        for label_to_import in raw_labels_to_import_for_subnet:
            if label_to_import in current_labels_for_subnet:
                logger.debug(f"The subnet {subnet} is already labeled with the label '{label_to_import.key}: "
                             f"{label_to_import.value}'")
            else:
                labels_to_import[subnet].append(label_to_import)

        duplicate_labels_for_subnet = \
            locate_duplicate_labels_for_subnet(subnet, raw_labels_to_import_for_subnet, current_labels_for_subnet)
        if duplicate_labels_for_subnet:
            duplicate_labels[subnet] = duplicate_labels_for_subnet

    return labels_to_import, duplicate_labels

In [58]:
def process_raw_labels_to_import_for_name_criteria(raw_labels_to_import_for_name_criteria, centra_labels):
    """
    Process the raw labels to import for name criteria. For each NameCriterion:
    1. Create a list of all the new labels that should be imported for it (= the name criteria is not already labeled in
     Centra with this label)
    2. Create a list of all the duplicate labels (multiple labels with the same key) it will have in case the labels
    import action will be performed.
    :param raw_labels_to_import_for_name_criteria: a dictionary mapping each NameCriterion to the list of it's
    labels to import
    :param centra_labels: dict containing all Centra labels ((key, value): label_obj)
    :return:
        labels_to_import - dict mapping each NameCriterion to it's labels to import
        duplicate_labels - dict mapping each NameCriterion to it's duplicate labels
    """
    labels_to_import = defaultdict(list)
    duplicate_labels = dict()
    current_labels_for_name_criteria = \
        map_current_labels_for_name_criteria(centra_labels, set(raw_labels_to_import_for_name_criteria.keys()))

    for name_criterion, raw_labels_to_import_for_name_criterion in raw_labels_to_import_for_name_criteria.items():
        current_labels_for_name_criterion = current_labels_for_name_criteria.get(name_criterion, list())
        for label_to_import in raw_labels_to_import_for_name_criterion:
            if label_to_import in current_labels_for_name_criterion:
                logger.debug(f"The name criterion {name_criterion} is already part of the criteria of the label "
                             f"'{label_to_import}'")
            else:
                labels_to_import[name_criterion].append(label_to_import)

        duplicate_labels_for_name_criterion = \
            locate_duplicate_labels_for_name_criterion(name_criterion, raw_labels_to_import_for_name_criterion,
                                                       current_labels_for_name_criterion)
        if duplicate_labels_for_name_criterion:
            duplicate_labels[name_criterion] = duplicate_labels_for_name_criterion

    return labels_to_import, duplicate_labels

In [59]:
def generate_import_and_update_actions_for_name_criteria(labels_to_import, duplicate_labels, allow_duplicates,
                                                         centra_labels, centra_labels_to_create,
                                                         centra_labels_to_update, actions_as_strings):
    """
    Process the labels to import and the duplicate labels for each name criterion:
     - Populate centra_labels_to_create with Centra labels that need to be created
     - Populate centra_labels_with_updates with Centra labels that need to be updated
     - Populate actions_as_strings with actions to be taken as string.
    :param actions_as_strings: list of actions to be taken as string  (i.e. "Adding the criterion 'Asset name starts
    with 'Ecomm'' to the label 'App: Ecomm'").
    :param centra_labels_to_update: a set of Labels pointing to the Centra labels that have updates
    (the actual updates are made on centra_labels dict)
    :param centra_labels_to_create: dictionary mapping a new Centra label to its criteria (asset_ids, subnets,
    name criteria)
    :param labels_to_import: a dict mapping each name criterion to a list of its labels to import
    :param duplicate_labels: a dict mapping each name criterion to a dict mapping each of its new labels to a list of
     the duplicate labels it has in Centra
    :param allow_duplicates: Boolean - Whether to allow duplicate labels or override them
    :param centra_labels: dict containing all Centra labels (Label: label_obj). Changes to labels that should be
    updated are performed on label objects in this dict
    :return: None
    """

    for name_criterion in labels_to_import:
        for label_to_import in labels_to_import[name_criterion]:
            action = f"Adding the criterion \"{name_criterion}\" to the criteria of the label '{label_to_import}'"
            # Import the label for the target by adding a criterion to an existing label or create a new label in case
            # the label does not exist in Centra yet
            if label_to_import in centra_labels:
                label_to_add_the_name_criterion_to = centra_labels[label_to_import]
                centra_labels_to_update.add(label_to_import)
            else:
                label_to_add_the_name_criterion_to = centra_labels_to_create[label_to_import]

            if "dynamic_criteria" not in label_to_add_the_name_criterion_to:
                label_to_add_the_name_criterion_to["dynamic_criteria"] = list()
            label_to_add_the_name_criterion_to["dynamic_criteria"].append({"field": "name",
                                                                           "op": name_criterion.criterion_type,
                                                                           "argument": name_criterion.name})

            # Mark duplicate labels for deletion
            if not allow_duplicates and label_to_import in duplicate_labels.get(name_criterion, dict()):
                duplicate_labels_string = ", ".join([f"'{label}'" for label in
                                                     duplicate_labels[name_criterion][label_to_import]])
                action += f", overriding the current label(s): {duplicate_labels_string}"
                for duplicate_label in duplicate_labels[name_criterion][label_to_import]:
                    label_to_remove_duplicate_from = centra_labels[duplicate_label]
                    centra_labels_to_update.add(duplicate_label)
                    current_dynamic_criteria = label_to_remove_duplicate_from["dynamic_criteria"]
                    new_dynamic_criteria = \
                        [criterion for criterion in current_dynamic_criteria if not
                            (criterion["op"] == name_criterion.criterion_type and
                             criterion["argument"] == name_criterion.name)]
                    label_to_remove_duplicate_from["dynamic_criteria"] = new_dynamic_criteria
                # remove the duplicate labels list so it wont be iterated over twice
                del duplicate_labels[name_criterion][label_to_import]

            actions_as_strings.append(action)

    # Mark for removal duplicate labels of labels that are mentioned in the CSV for the name criterion but not going
    # to be imported because the name criterion is already labeled with them in Centra
    if not allow_duplicates:
        for name_criterion in duplicate_labels:
            if not duplicate_labels[name_criterion]:
                continue
            for label_that_already_exist, duplicate_labels_list in duplicate_labels[name_criterion].items():
                duplicate_labels_string = ", ".join([f"'{label}'" for label in duplicate_labels_list])
                action = f"Overriding the label(s) {duplicate_labels_string} that are duplicates of the label " \
                         f"'{label_that_already_exist}', for the name criterion {name_criterion}"
                for duplicate_label in duplicate_labels_list:
                    label_to_remove_duplicate_from = centra_labels[duplicate_label]
                    centra_labels_to_update.add(duplicate_label)
                    current_dynamic_criteria = label_to_remove_duplicate_from["dynamic_criteria"]
                    new_dynamic_criteria = \
                        [criterion for criterion in current_dynamic_criteria if not
                            (criterion["op"] == name_criterion.criterion_type and
                             criterion["argument"] == name_criterion.name)]
                    label_to_remove_duplicate_from["dynamic_criteria"] = new_dynamic_criteria
                # remove the duplicate labels list so it wont be iterated over twice
                actions_as_strings.append(action)
    return

In [60]:
def print_summary_and_ask_for_confirmation_for_import_action(num_of_labels_to_import, num_of_labels_to_update,
                                                             actions_as_strings, assets_not_in_centra, env_name):
    """
    Print the actions summary and prompt the user for permission to execute the import and update actions.
    Abort if the permission was denied.
    :return: True if the user approved, False if the user did not approve
    """
    logger.info("--------------PRE EXECUTION SUMMARY---------------")
    if len(assets_not_in_centra) > 0:
        logger.warning(f"The following {len(assets_not_in_centra)} asset(s) were mentioned in the CSV but "
                       f"does not exist in Centra: ")
        for asset_name in assets_not_in_centra:
            logger.warning(asset_name)
        if not args.include_deleted_assets:
            logger.info("If the assets are in Centra in Deleted state, consider re running the script providing "
                        "the --include_deleted_assets flag")
        logger.info("------------------------------------------------------")
    if len(actions_as_strings) == 0:
        logger.info("No actions to take or log")
        sys.exit(0)

    logger.info(f"About to upload labels to the system {env_name}")
    if num_of_labels_to_import:
        logger.info(f"{num_of_labels_to_import} new labels will be created in Centra")
    if num_of_labels_to_update:
        logger.info(f"{num_of_labels_to_update} existing labels will be updated in Centra")
    logger.info("------------------------------------------------------")

    return print_action_summary_and_ask_for_confirmation(actions_as_strings)

In [61]:
def upload_new_labels_to_centra(gc_api, labels_to_create) -> None:
    """
    Format the labels to create in the way that Centra API expects them and upload them to Centra
    :param gc_api: RESTManagementAPI object
    :param labels_to_create: dictionary mapping a new label (key, value) to its criteria (asset_ids, subnets, ips)
    """
    labels_to_upload = list()
    for label_tuple, label_criteria in labels_to_create.items():
        key, value = label_tuple
        criteria = list()
        criteria += label_criteria.get("dynamic_criteria", list())
        criteria += label_criteria.get("equal_criteria", list())
        label_obj_to_api = {"key": key, "value": value, "criteria": criteria}
        labels_to_upload.append(label_obj_to_api)
    logger.info(f"Uploading {len(labels_to_upload)} label(s) to Centra")
    try:
        gc_api.bulk_create_labels(labels_to_upload)
    except ManagementAPITimeoutError:
        logger.error(f"The request for label creation has timed out")
        logger.info("Do not worry! This does not mean the labels were not uploaded to Centra, only that it "
                    "takes Centra some time to process the labels creation request as the request is big.\n"
                    "To make sure all labels were created successfully in Centra, wait 5 minutes and re run the import "
                    "command with the same input. The script should prompt that no action needs to be taken.\n"
                    "Alternatively, review the labels manually in Centra UI.")
    except ManagementAPIError as e:
        logger.error(f"Error while uploading new labels: {repr(e)}")


In [62]:
def is_subnet_or_ip(s):
    """
    Check if a string is a legal CIDR notated subnet or ip address.
    :return Boolean
    """
    return re.match(IP_REGEX, s)

In [68]:
try:
    from configparser import NoOptionError, NoSectionError, ConfigParser, MissingSectionHeaderError
except ImportError:
    # Probably no python 3 installed. Will catch it during validate_python_version and exit
    pass

#logger = initiate_logger(verbose=args.verbose, log_file_path=args.log_file_path)
logger = initiate_logger(verbose=True, log_file_path='test.log')
logger.debug(f"Running script")
raw_labels_to_import = parse_labels_csv('file_name.csv', True,True)
raw_labels_to_import_for_assets, raw_labels_to_import_for_subnets, labels_to_import_for_name_criteria = sort_labels_for_targets_by_target_types(raw_labels_to_import, True)
with initiate_gc_api() as gc_api:
     centra_assets_raw = get_centra_assets(gc_api, include_deleted_assets=False)
    #centra_assets = {asset["id"]: asset for asset in centra_assets_raw}

ERROR:root:Could not connect to Centra API: ManagementAPIError("Error while handling POST request for uri /api/v3.0/authenticate: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))")
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/sample/ds/anaconda3/anaconda3/envs/py373/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/Users/sample/ds/anaconda3/anaconda3/envs/py373/lib/python3.7/site-packages/urllib3/connectionpool.py", line 376, in _make_request
    self._validate_conn(conn)
  File "/Users/sample/ds/anaconda3/anaconda3/envs/py373/lib/python3.7/site-packages/urllib3/connectionpool.py", line 994, in _validate_conn
    conn.connect()
  File "/Users/sample/ds/anaconda3/anaconda3/envs/py373/lib/python3.7/site-packages/urllib3/connection.py", line 360, in connect
    ssl_context=context,
  File "/Users/sample/ds/anaconda3/anaconda3/envs/py373/lib/python3.7/site-packages/urllib3/util/ssl_.py", line 383, in ssl_wrap_socket
    return context.wrap_socket(sock)
  File "/Users/sample/ds/anaconda3/anaconda3/envs/py373/lib/python3.7/ssl.py", line 412, in wrap_socket
    session=session
  File "/Users/sample/ds/anacond

TypeError: object of type 'NoneType' has no len()

In [26]:
raw_labels_to_import_for_assets

{'0': [Label(key='SYS_NAME', value='File1'),
  Label(key='OWNER', value='YES'),
  Label(key='CLASSIFICATION', value='NETWORK'),
  Label(key='CITY', value='JSQ'),
  Label(key='REGION', value='US')],
 '1': [Label(key='SYS_NAME', value='File1'),
  Label(key='OWNER', value='YES'),
  Label(key='CLASSIFICATION', value='server'),
  Label(key='CITY', value='lon'),
  Label(key='REGION', value='Uk')]}